In [ ]:
!pip install watermark

In [11]:
%load_ext watermark

In [12]:
%watermark -a "Rishit Kapoor 21BAI1046" -u -d -v -m

Author: Rishit Kapoor 21BAI1046

Last updated: 2024-02-02

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.58+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



## Classifying Cats and Dogs with Data Augmentation

In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [16]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Define the classes you want to classify
dog_class = 5
cat_class = 3

# Filter training and test data for dog and cat classes only
train_mask = (y_train[:, 0] == cat_class) | (y_train[:, 0] == dog_class)
test_mask = (y_test[:, 0] == cat_class) | (y_test[:, 0] == dog_class)

x_train = x_train[train_mask].astype('float32') / 255
x_test = x_test[test_mask].astype('float32') / 255

y_train = tf.keras.utils.to_categorical((y_train[train_mask, 0] == cat_class).astype(int), 2)
y_test = tf.keras.utils.to_categorical((y_test[test_mask, 0] == cat_class).astype(int), 2)


# Let's build a CNN using Keras' Sequential capabilities
model_1 = Sequential()

# 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5), strides=(2, 2), padding='same', input_shape=x_train.shape[1:]))
model_1.add(Activation('relu'))

# Another 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5), strides=(2, 2)))
model_1.add(Activation('relu'))

# 2x2 max pooling reduces to 3 x 3 x 32
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Dropout(0.25))

# Flatten turns 3x3x32 into 288x1
model_1.add(Flatten())
model_1.add(Dense(512))
model_1.add(Activation('relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(2))  # Adjusted for binary classification
model_1.add(Activation('softmax'))

model_1.summary()

batch_size = 32

# initiate RMSprop optimizer
opt = tf.keras.optimizers.legacy.RMSprop(lr=0.0005, decay=1e-6)

# Let's train the model using RMSprop
model_1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
)

datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model_1.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
            steps_per_epoch=x_train.shape[0] // batch_size,
            epochs=5,
            validation_data=(x_test, y_test))

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 16, 16, 32)        2432      
                                                                 
 activation_8 (Activation)   (None, 16, 16, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 6, 6, 32)          25632     
                                                                 
 activation_9 (Activation)   (None, 6, 6, 32)          0         
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 3, 3, 32)          0         
 g2D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 3, 3, 32)  

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


312/312 [==============================] - 13s 38ms/step - loss: 0.6809 - accuracy: 0.5665 - val_loss: 0.6540 - val_accuracy: 0.6320
Epoch 2/5
312/312 [==============================] - 12s 38ms/step - loss: 0.6524 - accuracy: 0.6116 - val_loss: 0.6270 - val_accuracy: 0.6445
Epoch 3/5
312/312 [==============================] - 13s 41ms/step - loss: 0.6272 - accuracy: 0.6483 - val_loss: 0.6088 - val_accuracy: 0.6890
Epoch 4/5
312/312 [==============================] - 12s 38ms/step - loss: 0.6175 - accuracy: 0.6579 - val_loss: 0.5807 - val_accuracy: 0.6960
Epoch 5/5
312/312 [==============================] - 12s 37ms/step - loss: 0.6038 - accuracy: 0.6678 - val_loss: 0.5702 - val_accuracy: 0.7005
